In [1]:
!python -V
!pip show tensorflow

Python 3.6.5
Name: tensorflow
Version: 1.8.0
Summary: TensorFlow helps the tensors flow
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: c:\users\b\desktop\direct\env\lib\site-packages
Requires: absl-py, gast, wheel, protobuf, astor, tensorboard, six, grpcio, termcolor, numpy
Required-by: 


### Load the GC-GRNN model

In [2]:
import io
import argparse

import glob
import yaml
import numpy as np

import tensorflow as tf # mind the tf version
from lib.utils import load_graph_data
from model.gcnn_supervisor import GCNNSupervisor # requires Tensorflow 1.8 or 1.9

c:\users\b\desktop\adversarial_traffic_flow\adversarial_gc_grnn\env\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\b\desktop\adversarial_traffic_flow\adversarial_gc_grnn\env\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\b\desktop\adversarial_traffic_flow\adversarial_gc_grnn\env\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("q

In [3]:
best_model_dir = './best_model/'
name_splits = glob.glob(best_model_dir+'*.index')[0].split('.')
model_filename = '.'+name_splits[1] + '.'+ name_splits[2]
print("The model being used for predictions is:",model_filename)

The model being used for predictions is: ./best_model\models-325.1414-29274


In [4]:
#print("Both below are required only in training:")
predictions_file ="GCGRNN_volume_150.csv"
#print("The predictions at the end of this notebook will be available in :", predictions_file)

ground_truths_file = "y_truth_GCGRNN_volume_150.csv"
#print("The ground truth of those predictions will be available in :", ground_truths_file)

In [5]:
# Configuration YAML for the GC-GRNN model

configuration = {'base_dir' : 'results',
                 'log_level':'INFO',
                 'data':{'batch_size': 32,
                           'dataset_dir': './data/',
                           'graph_pkl_filename': './helper_files/adj_mat_volume.pkl',
                           'test_batch_size': 64,
                           'val_batch_size': 64},
                 'model':{'cl_decay_steps': 2000,
                           'horizon': 12,
                           'input_dim': 1,
                           'l1_decay': 0 ,
                           'num_nodes': 150,
                           'num_rnn_layers': 1,
                           'output_dim': 1,
                           'rnn_units': 128,
                           'seq_len': 12,
                           'use_curriculum_learning': True},
                'train':{'base_lr': 0.01,
                          'dropout': 0,
                          'epoch': 0,
                          'epochs': 300,
                          'epsilon': 1.0e-3,
                          'global_step': 0,
                          'lr_decay_ratio': 0.1,
                          'max_grad_norm': 5,
                          'max_to_keep': 100,
                          'min_learning_rate': 2.0e-06,
                          'model_filename': model_filename,
                          'optimizer': 'adam',
                          'patience': 50,
                          'steps': '[20]',
                          'test_every_n_epochs': 1,
                          'preds_file': predictions_file ,
                          'groundtruth_file':ground_truths_file}
                }
                
# Write YAML file
config_file_name = './best_model/config_for_pre_trained_gcnn.yaml'
with io.open(config_file_name, 'w', encoding='utf8') as outfile:
    yaml.dump(configuration, outfile, default_flow_style=False, allow_unicode=True)

In [6]:
# Helper
def run_gcnn(config_file, output_file, use_gpu = False ):
    
    # safely load the config yaml file
    with open(config_file, 'r') as f:
        try: 
            cfile = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)
    
    #with open(config_file) as f:
        #cfile = yaml.load(f)
    
    # some GPU config settings with TF        
    tf_config = tf.ConfigProto()
    if use_gpu:
        tf_config = tf.ConfigProto(device_count={'GPU': 0})
    tf_config.gpu_options.allow_growth = True
    
    # load the adjacency matrix graph pickle
    graph_pkl_filename = cfile['data']['graph_pkl_filename']
    _, _, adj_mx = load_graph_data(graph_pkl_filename)
    
    with tf.Session(config=tf_config) as sess:

        # Instantiate a supervisor with adjacency matrix and config
        supervisor = GCNNSupervisor(adj_mx=adj_mx, **cfile)

        #load the best-model 
        supervisor.load(sess, cfile['train']['model_filename'])

        # get model output
        outputs = supervisor.evaluate(sess)

        # save the predicsions to file
        np.savez_compressed(output_file, **outputs)
        print('Predictions saved as {}.'.format(output_file))

In [7]:
output_filename = './best_model/best_model_output.npz'

# If you do not have a GPU, set the use_gpu option to False
run_gcnn(config_file_name,output_filename, use_gpu = True)  

2021-03-14 11:39:57,473 - INFO - Log directory: results\GCNN_DDGF_h_12_128_lr_0.01_bs_32_0314113957/
2021-03-14 11:39:57,475 - INFO - {'adj_mx': array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), 'base_dir': 'results', 'data': {'batch_size': 32, 'dataset_dir': './data/', 'graph_pkl_filename': './helper_files/adj_mat_volume.pkl', 'test_batch_size': 64, 'val_batch_size': 64}, 'log_level': 'INFO', 'model': {'cl_decay_steps': 2000, 'horizon': 12, 'input_dim': 1, 'l1_decay': 0, 'num_nodes': 150, 'num_rnn_layers': 1, 'output_dim': 1, 'rnn_units': 128, 'seq_len': 12, 'use_curriculum_learning': True}, 'train': {'base_lr': 0.01, 'dropout': 0, 'epoch': 0, 'epochs': 300, 'epsilon': 0.001, 'global_step': 0, 'groundtruth_file': 'y_truth_GCGRNN_volume_150.csv', 'lr_decay_ratio': 0.1, 'max_grad_nor